In [1]:
import Util
import FitnessFunction

Util.moduleClasses(FitnessFunction)

{'DeceptiveStepTrap': FitnessFunction.DeceptiveStepTrap,
 'DeceptiveTrap': FitnessFunction.DeceptiveTrap,
 'FitnessFunction': FitnessFunction.FitnessFunction,
 'NearestNeighborNK': FitnessFunction.NearestNeighborNK,
 'TestFitnessFunction': FitnessFunction.TestFitnessFunction}

In [1]:
c1 = (1, 2)
c2 = (3, 4)
c1 + c2

(1, 2, 3, 4)

In [3]:
def common_elements(list1, list2):
    return list(set(list1) & set(list2))

# Ví dụ
list1 = [1, 2, 3, 4, 5]
list2 = [4, 5, 6, 7, 8]
result = common_elements(list1, list2)
result


[4, 5]